In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from tensorflow.keras.layers.experimental.preprocessing import CategoryEncoding
from tensorflow.keras.layers.experimental.preprocessing import StringLookup
print('相关库加载完成')

In [ ]:
dataframe = pd.read_csv('../input/nasa-cnc/nasa_learn.csv')
dataframe

In [ ]:
val_dataframe = dataframe.sample(frac=0.01, random_state=1337)
train_dataframe = dataframe.drop(val_dataframe.index)

print(
    "Using %d samples for training and %d for validation"
    % (len(train_dataframe), len(val_dataframe))
)

In [ ]:
def dataframe_to_dataset(dataframe):
    dataframe = dataframe.copy()
    labels = dataframe.pop("VB")
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    ds = ds.shuffle(buffer_size=len(dataframe))
    return ds


train_ds = dataframe_to_dataset(train_dataframe)
val_ds = dataframe_to_dataset(val_dataframe)

In [ ]:
for x, y in train_ds.take(1):
    print("Input:", x)
    print("Target:", y)

In [ ]:
train_ds = train_ds.batch(32)
val_ds = val_ds.batch(32)

In [ ]:
def encode_numerical_feature(feature, name, dataset):
    # 为我们的功能创建一个归一化层
    normalizer = Normalization()

    # 准备仅产生我们功能的数据集了解数据统计
    feature_ds = dataset.map(lambda x, y: x[name])
    feature_ds = feature_ds.map(lambda x: tf.expand_dims(x, -1))

    # 了解数据统计
    normalizer.adapt(feature_ds)

    # 标准化输入功能
    encoded_feature = normalizer(feature)
    return encoded_feature

def encode_integer_categorical_feature(feature, name, dataset):
    # 为我们的整数索引创建CategoryEncoding
    encoder = CategoryEncoding(output_mode="binary")

    # 准备仅产生我们功能的数据集
    feature_ds = dataset.map(lambda x, y: x[name])
    feature_ds = feature_ds.map(lambda x: tf.expand_dims(x, -1))

    # 了解可能的索引空间
    encoder.adapt(feature_ds)

    # 将一键编码应用于我们的索引
    encoded_feature = encoder(feature)
    return encoded_feature

In [ ]:
# 整数型分类特征值
DOC = keras.Input(shape=(1,), name="DOC", dtype="int64")
feed = keras.Input(shape=(1,), name="feed", dtype="int64")
material = keras.Input(shape=(1,), name="material", dtype="int64")

In [ ]:
# 连续特征值
smcDC = keras.Input(shape=(1,), name="smcDC")
vib_table = keras.Input(shape=(1,), name="vib_table")
vib_spindle = keras.Input(shape=(1,), name="vib_spindle")
AE_table = keras.Input(shape=(1,), name="AE_table")
AE_spindle = keras.Input(shape=(1,), name="AE_spindle")

In [ ]:
all_inputs = [
    DOC,
    feed,
    material,
    smcDC,
    vib_table,
    vib_spindle,
    AE_table,
    AE_spindle,
]

In [ ]:
# 整数分类特征
DOC_encoded = encode_integer_categorical_feature(DOC, "DOC", train_ds)
feed_encoded = encode_integer_categorical_feature(feed, "feed", train_ds)
material_encoded = encode_integer_categorical_feature(material, "material", train_ds)

In [ ]:
# 连续值特征
smcDC_encoded = encode_numerical_feature(smcDC, "smcDC", train_ds)
vib_table_encoded = encode_numerical_feature(vib_table, "vib_table", train_ds)
vib_spindle_encoded = encode_numerical_feature(vib_spindle, "vib_spindle", train_ds)
AE_table_encoded = encode_numerical_feature(AE_table, "AE_table", train_ds)
AE_spindle_encoded = encode_numerical_feature(AE_spindle, "AE_spindle", train_ds)

In [ ]:
all_features = layers.concatenate(
    [
        DOC_encoded,
        feed_encoded,
        material_encoded,
        smcDC_encoded,
        vib_table_encoded,
        vib_spindle_encoded,
        AE_table_encoded,
        AE_spindle_encoded,
    ]
)

In [ ]:
x = layers.Dense(32, activation="relu")(all_features)
x = layers.Dropout(0.5)(x)
output = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(all_inputs, output)
model.compile("adam", "binary_crossentropy", metrics=["accuracy"])

In [ ]:
model.fit(train_ds, epochs=200, validation_data=val_ds)